In [1]:
#Run cell to mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# install package to have access to custom functions
%pip install /content/drive/MyDrive/EMG_gestures/ --use-feature=in-tree-build

Processing ./drive/MyDrive/EMG_gestures
  Created wheel for EMG-gestures: filename=EMG_gestures-0.1.0-py3-none-any.whl size=38021 sha256=bba239ff5a8873d395f48f79b4bff995bc64bf0a9d3dcde1d6f1136a16148181
  Stored in directory: /tmp/pip-ephem-wheel-cache-_75gq0q0/wheels/a2/b7/61/2147fa082a9e51bef5dcc38dd3f0898fe0554d62203c0e383e
Successfully built EMG-gestures


In [3]:
#import necessary packages

#our workhorses
import numpy as np
import pandas as pd
import scipy

#to visualize
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
#style params for figures
sns.set(font_scale = 2)
plt.style.use('seaborn-white')
plt.rc("axes", labelweight="bold")
from IPython.display import display, HTML

#to load files
import os
import sys
import h5py

#import cusotm functions
from EMG_gestures.utils import *
from EMG_gestures.analysis import get_trained_rnn_model, evaluate_trained_rnn, rnn_xsubject_test


In [ ]:
#define where the data files are located
data_folder = '/content/drive/MyDrive/EMG_gestures/EMG_data/'
results_folder = '/content/drive/MyDrive/EMG_gestures/results_data/single_subject_training/RNN/'

nsubjects = 36


# User-defined parameters
lo_freq = 20 #lower bound of bandpass filter
hi_freq = 450 #upper bound of bandpass filter

win_size = 100 #define window size over which to compute time-domain features
step = win_size #keeping this parameter in case we want to re-run later with some overlap


#for RNN training
epochs = 40
batch_size = 2


nreps = 10
nsets_training = 10
exclude = [0,7]#labels to exclude
#performance metrics
score_list = ['f1','accuracy']

src_subject_id = 1
for src_subject_id in range(15,nsubjects+1):

    results_df = rnn_xsubject_test(data_folder, src_subject_id, nsubjects, nreps, lo_freq, hi_freq, win_size, step, exclude, score_list,\
                            nsets_training, epochs = epochs, batch_size = batch_size, mv = None, permute = False)

    results_df['Shuffled'] = False

    perm_results_df = rnn_xsubject_test(data_folder, src_subject_id, nsubjects, nreps, lo_freq, hi_freq, win_size, step, exclude, score_list,\
                            nsets_training, epochs = epochs, batch_size = batch_size, mv =None, permute = True)

    perm_results_df['Shuffled'] = True

    #concatenate

    results_df = pd.concat([results_df, perm_results_df]).reset_index(drop = True)
    results_df['Train_Subject'] = src_subject_id

    # #save results to file
    results_fn = 'subject_%02d_across_subject_results.h5'%(src_subject_id)
    results_df.to_hdf(os.path.join(results_folder,results_fn), key='results_df', mode='w')


Streaming output truncated to the last 5000 lines.
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/27
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/28
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/29
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/30
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/31
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/32
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/33
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/34
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/35
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/36
Evaluate Model
Subject 15|Rep 5
Training Model
Evaluate Model on Trained Data
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/01
Evaluate Model
Target Subject :/

In [7]:
results_df.groupby(['Shuffled','Type']).mean()

Rep  Test_Subject  f1_score  accuracy_score  Train_Subject
Shuffled Type                                                             
False    Test   350           350       350             350            350
         Train   10            10        10              10             10
True     Test   350           350       350             350            350
         Train   10            10        10              10             10

In [6]:
results_df

,Type,Rep,Test_Subject,f1_score,accuracy_score,Shuffled,Train_Subject
0,Test,1,2,0.278142,0.330144,False,1
1,Test,1,3,0.783836,0.800000,False,1
2,Test,1,4,0.774795,0.784360,False,1
3,Test,1,5,0.624997,0.664122,False,1
4,Test,1,6,0.499904,0.561224,False,1
...,...,...,...,...,...,...,...
715,Train,6,1,0.745650,0.752570,True,1
716,Train,7,1,0.747254,0.750467,True,1
717,Train,8,1,0.769442,0.773832,True,1
718,Train,9,1,0.672894,0.677570,True,1
